# API PROMPTING (OMG EXCITING)

In [ ]:
from dotenv import load_dotenv
import os
import csv
import random
import time
from collections import Counter
from openai import OpenAI

load_dotenv()  # reads .env file automatically

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

ModuleNotFoundError: No module named 'openai.error'

In [ ]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

response = client.chat.completions.create(
    model="gpt-5",
    messages=[{"role": "user", "content": "Say hello"}]
)
print(response.choices[0].message.content)


Hello!


In [7]:

models = client.models.list()
for m in models.data:
    print(m.id)

gpt-5-search-api
gpt-5-search-api-2025-10-14
dall-e-2
gpt-4-turbo-2024-04-09
gpt-audio
gpt-4o-mini-tts
gpt-4-turbo
gpt-realtime
gpt-realtime-2025-08-28
gpt-4.1-mini
gpt-4.1-mini-2025-04-14
gpt-3.5-turbo-1106
gpt-4-0125-preview
gpt-4-turbo-preview
gpt-4-0613
gpt-4
gpt-4.1
gpt-4.1-2025-04-14
text-embedding-ada-002
dall-e-3
gpt-4.1-nano
gpt-3.5-turbo-instruct-0914
o1-2024-12-17
gpt-4.1-nano-2025-04-14
gpt-3.5-turbo-16k
gpt-audio-2025-08-28
gpt-3.5-turbo-instruct
gpt-4o-2024-11-20
gpt-4o-2024-05-13
gpt-4o-mini-search-preview
gpt-4o-mini-search-preview-2025-03-11
gpt-4o-search-preview
omni-moderation-latest
o1-pro
o1-pro-2025-03-19
gpt-4o-search-preview-2025-03-11
text-embedding-3-small
o1-mini-2024-09-12
gpt-4o-2024-08-06
o1
gpt-4o-mini-2024-07-18
gpt-4o-mini
gpt-4o-mini-audio-preview
gpt-image-1-mini
gpt-5-mini
gpt-image-1
gpt-5-chat-latest
gpt-5-mini-2025-08-07
omni-moderation-2024-09-26
gpt-5
gpt-5-nano-2025-08-07
gpt-4o-audio-preview-2024-12-17
gpt-5-nano
tts-1-1106
tts-1-hd-1106
tts-1

In [17]:
import os
import csv
import time

# Setting paths
if "__file__" in globals():
    script_dir = os.path.dirname(os.path.abspath(__file__))
else:
    script_dir = os.getcwd()

base_dir = os.path.abspath(os.path.join(script_dir, ".."))

season1_path = os.path.join(base_dir, "data_output", "season1_chunk.txt")
summaries_path = os.path.join(base_dir, "data_output", "summaries_MEETING", "scene_summaries_MEETING.csv")
output_path = os.path.join(base_dir, "data_output", "GPT5_scenes.csv")

# Feeding baseline info
with open(season1_path, "r", encoding="utf-8") as f:
    season1_text = f.read().strip()

# GPT call with rate-limit handling
def call_gpt_with_retry(prompt, max_retries=5):
    delay = 3  # initial wait in seconds
    for attempt in range(max_retries):
        try:
            completion = client.chat.completions.create(
                model="gpt-5-search-api",
                messages=[{"role": "user", "content": prompt}],
                max_tokens=200
            )
            return completion.choices[0].message.content
        except Exception as e:  # catch all exceptions
            # Check if this is a rate limit error
            if "rate limit" in str(e).lower() or "429" in str(e):
                wait_time = delay
                print(f"Rate limit hit. Waiting {wait_time:.2f}s before retry (attempt {attempt+1}/{max_retries})...")
                time.sleep(wait_time)
                delay *= 2  # exponential backoff
            else:
                raise e
    raise Exception("Max retries reached due to rate limits.")

# Build the prompt
def build_prompt(row):
    return f"""Ignore all previous messages.
You have access to the following baseline text from Season 1:
---
{season1_text}
---
Now make a manuscript for a scene in Season 2.
Ensure the scene sticks to the actions as explained by the summary.
Keep the tone and characters true to the first season.
Do your best to keep the number of lines similar to the number given.

Summary: {row['summary']}
Number of lines: {row['num_lines']}
"""

# Process CSV
results = []
with open(summaries_path, newline="", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        prompt = build_prompt(row)
        response = call_gpt_with_retry(prompt)
        results.append({
            "num_lines": row["num_lines"],
            "summary": row["summary"],
            "scene": response
        })
        # Optional small sleep to reduce bursts
        time.sleep(0.5)

# Write output CSV
with open(output_path, "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=["num_lines", "summary", "scene"])
    writer.writeheader()
    writer.writerows(results)


Rate limit hit. Waiting 3.00s before retry (attempt 1/5)...
Rate limit hit. Waiting 3.00s before retry (attempt 1/5)...
Rate limit hit. Waiting 3.00s before retry (attempt 1/5)...
Rate limit hit. Waiting 3.00s before retry (attempt 1/5)...
Rate limit hit. Waiting 3.00s before retry (attempt 1/5)...
Rate limit hit. Waiting 3.00s before retry (attempt 1/5)...
Rate limit hit. Waiting 3.00s before retry (attempt 1/5)...
Rate limit hit. Waiting 3.00s before retry (attempt 1/5)...
Rate limit hit. Waiting 3.00s before retry (attempt 1/5)...
Rate limit hit. Waiting 3.00s before retry (attempt 1/5)...
Rate limit hit. Waiting 3.00s before retry (attempt 1/5)...
Rate limit hit. Waiting 3.00s before retry (attempt 1/5)...
Rate limit hit. Waiting 3.00s before retry (attempt 1/5)...
Rate limit hit. Waiting 3.00s before retry (attempt 1/5)...
Rate limit hit. Waiting 3.00s before retry (attempt 1/5)...
Rate limit hit. Waiting 3.00s before retry (attempt 1/5)...
Rate limit hit. Waiting 3.00s before ret

In [14]:
def call_gpt(prompt):
    completion = client.chat.completions.create(
        model="gpt-4.1-mini",     # INPUT THE MODEL TO BE USED HERE
        messages=[{"role": "user", "content": prompt}],
        max_tokens=200
    )
    return completion.choices[0].message.content

def build_prompt(row):
    return f"""Ignore all previous messages.
Answer only this context and nothing else.

ID: {row['ID']}
Context: {row['Context']}
"""

with open("context_ID.csv", newline="", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    results = []
    for row in reader:
        prompt = build_prompt(row)
        response = call_gpt(prompt)
        results.append({"ID": row["ID"], "Context": row["Context"], "response": response})

with open("GPT_responses.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=["ID", "Context", "response"])
    writer.writeheader()
    writer.writerows(results)